In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as op
import math
import torch
import re

In [3]:
data = pandas.read_csv('./data/Power_pReLU.csv')
data_split = data.iloc[:, 1:]
data_split

,"power (RH2=77500,RL2=875000,RD2=5750,Rbase=47500,w=0.00065,L=0.00011) Y","power (RH2=32500,RL2=1625000,RD2=15250,Rbase=82500,w=0.00035,L=9e-05) Y","power (RH2=43750,RL2=1062500,RD2=12875,Rbase=91250,w=0.000425,L=8.5e-05) Y","power (RH2=88750,RL2=1812500,RD2=3375,Rbase=56250,w=0.000725,L=0.000105) Y","power (RH2=66250,RL2=687500,RD2=17625,Rbase=73750,w=0.000575,L=0.000115) Y","power (RH2=21250,RL2=1437500,RD2=8125,Rbase=38750,w=0.000275,L=9.5e-05) Y","power (RH2=26875,RL2=968750,RD2=18812,Rbase=60625,w=0.000538,L=9.2e-05) Y","power (RH2=71875,RL2=1718750,RD2=9312,Rbase=95625,w=0.000238,L=0.000112) Y","power (RH2=94375,RL2=593750,RD2=14062,Rbase=43125,w=0.000388,L=0.000102) Y","power (RH2=49375,RL2=1343750,RD2=4562,Rbase=78125,w=0.000688,L=8.2e-05) Y",...,"power (RH2=21662,RL2=1600922,RD2=1859,Rbase=40634,w=0.000596,L=9.8e-05) Y","power (RH2=27287,RL2=944672,RD2=12547,Rbase=62509,w=0.000259,L=9e-05) Y","power (RH2=72287,RL2=1694672,RD2=3047,Rbase=97509,w=0.000559,L=0.00011) Y","power (RH2=94787,RL2=569672,RD2=17297,Rbase=45009,w=0.000709,L=0.0001) Y","power (RH2=49787,RL2=1319672,RD2=7797,Rbase=80009,w=0.000409,L=8e-05) Y","power (RH2=38537,RL2=757172,RD2=5422,Rbase=71259,w=0.000484,L=9.5e-05) Y","power (RH2=83537,RL2=1507172,RD2=14922,Rbase=36259,w=0.000784,L=0.000115) Y","power (RH2=61037,RL2=1132172,RD2=10172,Rbase=88759,w=0.000634,L=0.000105) Y","power (RH2=16037,RL2=1882172,RD2=19672,Rbase=53759,w=0.000334,L=8.5e-05) Y","power (RH2=10000,RL2=500000,RD2=1000,Rbase=30000,w=0.0002,L=8e-05) Y"
0,0.000089,0.000051,0.000048,0.000073,0.000061,0.000106,0.000070,0.000044,0.000101,0.000054,...,0.000101,0.000068,0.000043,0.000102,0.000053,0.000061,0.000113,0.000049,0.000077,0.000141
1,0.000088,0.000050,0.000047,0.000073,0.000060,0.000105,0.000069,0.000044,0.000100,0.000054,...,0.000100,0.000067,0.000043,0.000101,0.000052,0.000061,0.000112,0.000048,0.000076,0.000140
2,0.000087,0.000050,0.000047,0.000072,0.000059,0.000104,0.000069,0.000043,0.000099,0.000053,...,0.000099,0.000067,0.000043,0.000099,0.000052,0.000060,0.000111,0.000048,0.000075,0.000138
3,0.000086,0.000049,0.000046,0.000071,0.000059,0.000103,0.000068,0.000043,0.000098,0.000053,...,0.000098,0.000066,0.000042,0.000098,0.000051,0.000059,0.000110,0.000047,0.000074,0.000137
4,0.000085,0.000049,0.000046,0.000070,0.000058,0.000102,0.000067,0.000042,0.000097,0.000052,...,0.000097,0.000065,0.000042,0.000097,0.000051,0.000059,0.000109,0.000047,0.000073,0.000136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,0.000235,0.000139,0.000155,0.000262,0.000146,0.000185,0.000164,0.000101,0.000173,0.000274,...,0.000259,0.000142,0.000180,0.000191,0.000184,0.000197,0.000227,0.000180,0.000155,0.000164
397,0.000238,0.000140,0.000157,0.000265,0.000148,0.000187,0.000166,0.000102,0.000175,0.000277,...,0.000262,0.000143,0.000182,0.000193,0.000186,0.000199,0.000229,0.000182,0.000157,0.000166
398,0.000240,0.000142,0.000158,0.000269,0.000149,0.000189,0.000168,0.000104,0.000177,0.000280,...,0.000265,0.000145,0.000184,0.000196,0.000188,0.000202,0.000232,0.000185,0.000158,0.000168
399,0.000243,0.000144,0.000160,0.000272,0.000151,0.000192,0.000170,0.000105,0.000179,0.000284,...,0.000268,0.000146,0.000186,0.000198,0.000190,0.000204,0.000234,0.000187,0.000160,0.000169


In [4]:
datalist=data_split.columns.values.tolist()
print(datalist)
print(len(datalist))

['power (RH2=77500,RL2=875000,RD2=5750,Rbase=47500,w=0.00065,L=0.00011) Y', 'power (RH2=32500,RL2=1625000,RD2=15250,Rbase=82500,w=0.00035,L=9e-05) Y', 'power (RH2=43750,RL2=1062500,RD2=12875,Rbase=91250,w=0.000425,L=8.5e-05) Y', 'power (RH2=88750,RL2=1812500,RD2=3375,Rbase=56250,w=0.000725,L=0.000105) Y', 'power (RH2=66250,RL2=687500,RD2=17625,Rbase=73750,w=0.000575,L=0.000115) Y', 'power (RH2=21250,RL2=1437500,RD2=8125,Rbase=38750,w=0.000275,L=9.5e-05) Y', 'power (RH2=26875,RL2=968750,RD2=18812,Rbase=60625,w=0.000538,L=9.2e-05) Y', 'power (RH2=71875,RL2=1718750,RD2=9312,Rbase=95625,w=0.000238,L=0.000112) Y', 'power (RH2=94375,RL2=593750,RD2=14062,Rbase=43125,w=0.000388,L=0.000102) Y', 'power (RH2=49375,RL2=1343750,RD2=4562,Rbase=78125,w=0.000688,L=8.2e-05) Y', 'power (RH2=38125,RL2=781250,RD2=6938,Rbase=69375,w=0.000762,L=9.8e-05) Y', 'power (RH2=83125,RL2=1531250,RD2=16438,Rbase=34375,w=0.000462,L=0.000118) Y', 'power (RH2=60625,RL2=1156250,RD2=2188,Rbase=86875,w=0.000312,L=0.000108)

In [5]:
def extract_values(data_list):
    patterns = [
        r"RH2=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"RL2=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"RD2=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"Rbase=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"w=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"L=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))"
    ]
    
    matrix = []
    
    for data in data_list:
        row = []
        for pattern in patterns:
            match = re.search(pattern, data)
            if match:
                value = match.group(1)
                if 'e' in value or '.' in value: 
                    row.append(float(value))
                else:
                    row.append(int(value))
            else:
                row.append(None)
        matrix.append(row)
    return matrix

result = extract_values(datalist)

In [6]:
result_temp = torch.tensor(result)
result_temp.max(0)[0]

tensor([9.9989e+04, 1.9998e+06, 1.9998e+04, 9.9991e+04, 8.0000e-04, 1.2000e-04])

In [7]:
result_temp.min(0)[0]

tensor([1.0000e+04, 5.0000e+05, 1.0000e+03, 3.0000e+04, 2.0000e-04, 8.0000e-05])

In [8]:
torch.save(result_temp, './data/pReLU_RT.data')

In [9]:
rh = result_temp[:, 0].view(-1,1)
rl = result_temp[:, 1].view(-1,1)
rd = result_temp[:, 2].view(-1,1)
rb = result_temp[:, 3].view(-1,1)
w  = result_temp[:, 4].view(-1,1)
l  = result_temp[:, 5].view(-1,1)
ratio_1 = rh/rl
ratio_2 = w/l

rh.shape, rl.shape, rd.shape, rb.shape, w.shape, l.shape, ratio_1.shape, ratio_2.shape

(torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]))

In [10]:
C = torch.cat([rh, rl, rd, rb, w, l, ratio_1, ratio_2], dim=1)
C.shape

torch.Size([9999, 8])

In [11]:
torch.save(C, './data/pReLU_RT_ratio.data')